```python
# TODO: 找几个描述统计量卷可视化
# TODO: 如果需要，做多元独立性检验
```

In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
import statsmodels.api as sm
from statsmodels.tools import add_constant

In [154]:
df = pd.read_csv('cleaned_data.csv', index_col=0)
df.head()

,gender,major,grade,origin,money,1010,1011,1012,1013,0010,...,2102,2103,1110,1111,1112,1113,1000,1001,1002,1003
0,1,3,2,0,3000,0.435364,0.00000,0.564636,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0
1,1,1,2,2,4503,0.000000,0.00000,0.272892,0.727108,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,1.0
2,2,2,2,2,2134,0.000000,0.00000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,1.0
3,2,2,2,1,2604,0.269976,0.32322,0.406804,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.432907,0.567093,0.0
4,1,2,1,0,3000,0.399108,0.26753,0.333363,0.000000,0.189496,...,0.0,0.0,0.0,1.0,0.0,0.0,0.332225,0.367165,0.300610,0.0


## ANOVA

In [155]:
def anova(df: pd.DataFrame, option: int):
    '''
    对某一个交通工具的softmax概率做anova
    '''
    zero_cols = df.columns[df.columns.str.endswith(str(option))]

    # 将所有0结尾的列转换为长格式
    zero_df = df[zero_cols].melt(var_name='category', value_name='prob')
    S = ['Distance', 'Weather', 'Constraint']
    for i, s in enumerate(S):
        zero_df[s] = zero_df['category'].str[i]

    zero_df.drop('category', axis=1, inplace=True)
    zero_df[S] = zero_df[S].astype(int)

    formula = f'prob ~ C({S[0]}) + C({S[1]}) + C({S[2]}) + C({S[0]}):C({S[1]}) + C({S[0]}):C({S[2]}) + C({S[1]}):C({S[2]}) + C({S[0]}):C({S[1]}):C({S[2]})'
    model = ols(formula, zero_df).fit()
    anova_res = anova_lm(model)

    return anova_res

In [156]:
anova(df, 0)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,25.704493,12.852246,158.374513,1.260813e-60
C(Weather),1.0,0.267547,0.267547,3.296901,6.970422e-02
C(Constraint),1.0,2.948794,2.948794,36.337135,2.316561e-09
C(Distance):C(Weather),2.0,1.691500,0.845750,10.421931,3.307035e-05
C(Distance):C(Constraint),2.0,3.676023,1.838012,22.649284,2.375212e-10
C(Weather):C(Constraint),1.0,0.081415,0.081415,1.003256,3.167614e-01
C(Distance):C(Weather):C(Constraint),2.0,0.262162,0.131081,1.615275,1.993441e-01
Residual,1020.0,82.773997,0.081151,NaN,NaN


In [157]:
anova(df, 1)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,4.767242,2.383621,25.541614,1.500535e-11
C(Weather),1.0,9.788587,9.788587,104.889297,1.693486e-23
C(Constraint),1.0,10.376253,10.376253,111.186411,9.574503e-25
C(Distance):C(Weather),2.0,4.817967,2.408984,25.813386,1.158442e-11
C(Distance):C(Constraint),2.0,1.849894,0.924947,9.911240,5.456252e-05
C(Weather):C(Constraint),1.0,3.878330,3.878330,41.558121,1.762262e-10
C(Distance):C(Weather):C(Constraint),2.0,5.323056,2.661528,28.519520,8.872590e-13
Residual,1020.0,95.189494,0.093323,NaN,NaN


In [158]:
anova(df, 2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,25.264952,12.632476,186.241284,1.130593e-69
C(Weather),1.0,6.935398,6.935398,102.248959,5.678035e-23
C(Constraint),1.0,0.268616,0.268616,3.960224,4.685411e-02
C(Distance):C(Weather),2.0,3.491657,1.745829,25.738849,1.243622e-11
C(Distance):C(Constraint),2.0,0.279014,0.139507,2.056757,1.283980e-01
C(Weather):C(Constraint),1.0,1.799275,1.799275,26.526811,3.118443e-07
C(Distance):C(Weather):C(Constraint),2.0,1.794274,0.897137,13.226539,2.133094e-06
Residual,1020.0,69.185118,0.067829,NaN,NaN


In [159]:
anova(df, 3)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,4.726694,2.363347,53.395220,8.832411e-23
C(Weather),1.0,0.000488,0.000488,0.011027,9.163907e-01
C(Constraint),1.0,0.971667,0.971667,21.952929,3.172332e-06
C(Distance):C(Weather),2.0,0.138526,0.069263,1.564866,2.096177e-01
C(Distance):C(Constraint),2.0,0.727463,0.363732,8.217806,2.880719e-04
C(Weather):C(Constraint),1.0,0.117405,0.117405,2.652526,1.036943e-01
C(Distance):C(Weather):C(Constraint),2.0,0.266390,0.133195,3.009280,4.976532e-02
Residual,1020.0,45.146627,0.044261,NaN,NaN


## Multinomial Logistic Regression

In [160]:
properties = ['gender', 'major', 'grade', 'origin', 'money']
df2 = df[properties].copy()
# 把money转为zscore
df2['money'] = (df2['money'] - df2['money'].mean()) / df2['money'].std()

situations = ['101', '001', '211', '000', '200', '011', '010', '110', '201', '210', '111', '100']
for i in situations:
    start, end = i + '0', i + '3'
    # In case of multiple occurrences of the maximum values, 
    # the indices corresponding to the first occurrence are returned.
    preferences = np.argmax(df.loc[:, start:end], axis=1)
    df2[i] = preferences
df2.head()

,gender,major,grade,origin,money,101,001,211,000,200,011,010,110,201,210,111,100
0,1,3,2,0,-0.026633,2,2,0,3,0,0,1,2,0,0,0,0
1,1,1,2,2,0.955822,3,3,1,3,1,1,3,3,1,1,1,3
2,2,2,2,2,-0.592705,2,2,1,3,0,2,3,2,1,0,1,3
3,2,2,2,1,-0.285483,2,2,0,3,0,1,3,3,1,0,1,2
4,1,2,1,0,-0.026633,0,2,1,2,0,3,1,0,1,1,1,1


In [161]:
df2 = df2.melt(
    id_vars=properties, 
    value_vars=situations, 
    var_name='situation', 
    value_name='preference'
)
S = ['Distance', 'Weather', 'Constraint']
for i, s in enumerate(S):
    df2[s] = df2['situation'].str[i]

df2.drop('situation', axis=1, inplace=True)
df2[S] = df2[S].astype(int)
df2.head()

,gender,major,grade,origin,money,preference,Distance,Weather,Constraint
0,1,3,2,0,-0.026633,2,1,0,1
1,1,1,2,2,0.955822,3,1,0,1
2,2,2,2,2,-0.592705,2,1,0,1
3,2,2,2,1,-0.285483,2,1,0,1
4,1,2,1,0,-0.026633,0,1,0,1


In [162]:
# 获取独热编码
one_hot_cols = ['gender', 'major', 'grade', 'origin', 'Distance']
df2 = pd.get_dummies(df2, columns=one_hot_cols, drop_first=True, dtype=int)
df2.head()

,money,preference,Weather,Constraint,gender_2,major_2,major_3,major_4,grade_2,grade_3,grade_4,origin_1,origin_2,Distance_1,Distance_2
0,-0.026633,2,0,1,0,0,1,0,1,0,0,0,0,1,0
1,0.955822,3,0,1,0,0,0,0,1,0,0,0,1,1,0
2,-0.592705,2,0,1,1,1,0,0,1,0,0,0,1,1,0
3,-0.285483,2,0,1,1,1,0,0,1,0,0,1,0,1,0
4,-0.026633,0,0,1,0,1,0,0,0,0,0,0,0,1,0


In [163]:
# 类型检查, 没有object就说明没有一列的类型是混合的
df2.dtypes

money         float64
preference      int64
Weather         int64
Constraint      int64
gender_2        int64
major_2         int64
major_3         int64
major_4         int64
grade_2         int64
grade_3         int64
grade_4         int64
origin_1        int64
origin_2        int64
Distance_1      int64
Distance_2      int64
dtype: object

In [164]:
X = df2.drop('preference', axis=1)
X = add_constant(X)
logit_model = sm.MNLogit(df2['preference'], X)
logit_res = logit_model.fit()
logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.912499
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:             preference   No. Observations:                 1032
Model:                        MNLogit   Df Residuals:                      987
Method:                           MLE   Df Model:                           42
Date:                Sun, 29 Dec 2024   Pseudo R-squ.:                  0.2776
Time:                        14:13:57   Log-Likelihood:                -941.70
converged:                       True   LL-Null:                       -1303.5
Covariance Type:            nonrobust   LLR p-value:                4.825e-125
================================================================================
preference=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.5448      0.329     -1.656      0.098      -1.190       0.100
money            0.3744      0.101      3.689      0.000       0.176       0.573
Weather          0.9064      0.172      5.260      0.000       0.569       1.244
Constraint       1.4965      0.172      8.723      0.000       1.160       1.833
gender_2         0.1824      0.201      0.908      0.364      -0.211       0.576
major_2          0.1234      0.203      0.609      0.543      -0.274       0.521
major_3         -0.2288      0.463     -0.494      0.621      -1.136       0.678
major_4         -0.3625      0.390     -0.930      0.352      -1.127       0.402
grade_2         -0.5091      0.194     -2.626      0.009      -0.889      -0.129
grade_3         -0.2125      0.360     -0.590      0.555      -0.918       0.493
grade_4         -0.7400      0.536     -1.380      0.168      -1.791       0.311
origin_1         0.1735      0.273      0.636      0.525      -0.361       0.708
origin_2         0.7581      0.303      2.499      0.012       0.163       1.353
Distance_1      -0.6037      0.241     -2.508      0.012      -1.076      -0.132
Distance_2      -1.1166      0.237     -4.710      0.000      -1.581      -0.652
--------------------------------------------------------------------------------
preference=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.2290      0.348      6.403      0.000       1.547       2.911
money           -0.0759      0.138     -0.552      0.581      -0.346       0.194
Weather         -1.3245      0.218     -6.082      0.000      -1.751      -0.898
Constraint       0.4247      0.210      2.025      0.043       0.014       0.836
gender_2        -0.3932      0.258     -1.526      0.127      -0.898       0.112
major_2         -0.1477      0.247     -0.597      0.550      -0.632       0.337
major_3         -0.3124      0.575     -0.543      0.587      -1.440       0.815
major_4         -0.2180      0.479     -0.455      0.649      -1.157       0.721
grade_2         -0.7277      0.242     -3.004      0.003      -1.202      -0.253
grade_3         -0.2230      0.432     -0.516      0.606      -1.070       0.624
grade_4          0.5600      0.573      0.977      0.329      -0.563       1.683
origin_1         1.0510      0.319      3.290      0.001       0.425       1.677
origin_2         0.3346      0.417      0.803      0.422      -0.482       1.152
Distance_1      -1.8897      0.238     -7.944      0.000      -2.356      -1.424
Distance_2      -5.6620      0.546    -10.373      0.000      -6.732      -4.592
--------------------------------------------------------------------------------
preference=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0102      0.501      0.020      0.984      -0.972       0.992
m